In [ ]:
import json
from datasets import Dataset

def load_data(file_path):
    # 데이터셋 로드
    with open(file_path, "r", encoding="utf-8") as f:
        dataset = json.load(f)

    # 데이터를 Trainer가 사용할 수 있도록 Dataset 형식으로 변환
    def generate_prompts(example):
        prompt_list = []
        for key, example in dataset.items():
            prompt_list.append(
                f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>다음 글을 요약해주세요:
{example['content']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{example['answer']}<|eot_id|>"""
            )
        return prompt_list

    # 프롬프트 리스트 생성
    prompts = generate_prompts(dataset)

    # Dataset으로 변환
    prompts_dataset = Dataset.from_list([{"input_text": prompt} for prompt in prompts])
    return prompts_dataset

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model

def train_model(tokenized_prompts, model, tokenizer):
    # DataCollator 정의 (MLM을 사용하지 않도록 설정)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # 학습 설정
    training_args = TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=1,  # 배치 크기 조정
        num_train_epochs=3,
        logging_steps=10,
        fp16=True,  # 16-bit 연산 활성화
        save_strategy="no",  # 모델 저장 전략 설정
    )

    # Trainer 설정
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_prompts,
        data_collator=data_collator  # DataCollatorForLanguageModeling 사용
    )

    # 학습 시작
    print(f"Model is on device: {model.device}")  # 모델이 할당된 GPU 디바이스 확인
    print("Starting training...")
    trainer.train()
    print("Training completed!")

def setup_model():
    BASE_MODEL = "meta-llama/Meta-Llama-3-8B-Instruct"
    
    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
    tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})  # pad_token 설정

    # 4bit 양자화 설정 - QLoRA로 해야 함
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 모델 로드 (양자화)
    model = AutoModelForCausalLM.from_pretrained(BASE_MODEL,
        quantization_config=bnb_config,
        device_map={"": 1}  # 두 번째 GPU로 할당
    )

    # PEFT 설정: LoRA 어댑터 추가
    lora_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.05,
        target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
        bias="none",
        task_type="CAUSAL_LM",
    )

    # LoRA 어댑터를 모델에 적용
    model = get_peft_model(model, lora_config)

    return model, tokenizer


In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# 데이터 로드
prompts_dataset = load_data("dataFromAIHUB.json")

# 모델 설정 (여기서 device_map을 통해 모델을 정확한 GPU에 로드)
model, tokenizer = setup_model()

# 데이터셋 토큰화
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["input_text"], padding="max_length", truncation=True, max_length=512
    )
    tokenized["labels"] = tokenized["input_ids"]  # labels 필드를 input_ids와 동일하게 설정
    return tokenized

# 데이터셋 토큰화
tokenized_prompts = prompts_dataset.map(tokenize_function, batched=True)

# 데이터셋 텐서 형식으로 설정
tokenized_prompts.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# DataCollator 정의 (MLM을 사용하지 않도록 설정)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,  # 배치 크기 조정
    num_train_epochs=3,
    logging_steps=10,
    fp16=True,  # 16-bit 연산 활성화
    save_strategy="no",  # 모델 저장 전략 설정
)

# 모델이 두 번째 GPU에 할당되었는지 확인
print(f"Model is on device: {model.device}")

# 메모리 비우기 (필요시 학습 전에)
torch.cuda.empty_cache()

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_prompts,
    data_collator=data_collator  # DataCollatorForLanguageModeling 사용
)

# 학습 시작
print("Starting training...")
trainer.train()
print("Training completed!")

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# 학습 전에 데이터를 model.device로 이동 (모델이 위치한 GPU로 이동)
tokenized_prompts = tokenized_prompts.to(model.device)  # 데이터가 모델과 동일한 GPU로 이동

# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,  # 배치 크기 조정
    num_train_epochs=3,
    logging_steps=10,
    fp16=True,  # 16-bit 연산 활성화
    save_strategy="no",  # 모델 저장 전략 설정
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_prompts,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)  # MLM을 사용하지 않음
)

# 학습 시작
print("Starting training...")
trainer.train()

print("Training completed!")


In [ ]:
# 학습이 완료된 모델을 사용하여 예측을 수행
text = "예시 텍스트"  # 예시 텍스트를 입력합니다.
inputs = tokenizer(text, return_tensors="pt").to(model.device)
outputs = model.generate(inputs['input_ids'], max_length=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

#################

In [2]:
import json
import torch
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

with open("dataFromAIHUB.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)
    
BASE_MODEL = "meta-llama/Meta-Llama-3-8B-Instruct"

# LoRA 설정 : 양자화된 모델에서 Adaptor를 붙여서 학습할 파라미터만 따로 구성함
lora_config = LoraConfig(
    r=8,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

# 4bit 양자화 설정 - QLoRA로 해야 함
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 모델 로드 (양자화)
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL,
                                             quantization_config=bnb_config,
                                             device_map={"": 1}  # 두 번째 GPU로 할당
                                            )

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
# tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})
tokenizer.padding_side = 'right'

# LoRA 적용
model = get_peft_model(model, lora_config)

def generate_prompts(example):
    prompt_list = []
    for key, example in dataset.items():
        prompt_list.append(
f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>다음 글을 요약해주세요:
{example['content']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{example['answer']}<|eot_id|>"""
        )
    return prompt_list

# 프롬프트 리스트 생성
prompts = generate_prompts(dataset)

print(prompts[0])

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
print(model.device)  # 모델이 할당된 장치 확인


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from datasets import Dataset
from transformers import Trainer, TrainingArguments, default_data_collator

# prompts 리스트를 Dataset 형식으로 변환
prompts_dataset = Dataset.from_list([{"input_text": prompt} for prompt in prompts])
# prompts_dataset = Dataset.from_list(prompts)

# 토큰화 함수 정의
def tokenize_function(examples):
    tokenized = tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].clone()  # 복사본을 labels로 추가
    return tokenized

# 토큰화 적용
tokenized_prompts = prompts_dataset.map(tokenize_function, batched=True)

# 모델을 GPU에 명시적으로 할당
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 토큰화된 데이터셋을 GPU에 할당하는 함수 추가
def move_to_device(batch):
    return {key: value.to(device) for key, value in batch.items()}

# 데이터셋의 모든 항목을 GPU로 이동
tokenized_prompts = tokenized_prompts.map(move_to_device, batched=True)

# 기존 Data Collator 정의를 default_data_collator로 대체
data_collator = default_data_collator

# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    fp16=True,  # 16-bit로 계산하여 메모리 절약
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_prompts,  # 토큰화된 데이터셋 사용
    # tokenizer=tokenizer,
    data_collator=data_collator  # 추가된 부분
)

# 학습 시작
trainer.train()

In [ ]:
from transformers import Trainer, TrainingArguments
import torch

# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,  # 배치 크기 조정
    num_train_epochs=3,
    logging_steps=10,
    fp16=True,  # 16-bit 연산 활성화
    save_strategy="no",  # 필요에 따라 변경 가능
    # CUDA 디바이스를 명시적으로 설정할 필요가 없지만,
    # torch.cuda.is_available()이 True일 경우 자동으로 GPU 사용
)

# Trainer 설정
trainer = Trainer(
    model=model,  # 모델이 두 번째 GPU에 이미 로드되어 있음
    args=training_args,
    train_dataset=tokenized_prompts,
    data_collator=data_collator  # DataCollatorForLanguageModeling 사용
)

# 모델이 두 번째 GPU에 잘 할당되었는지 확인
print(f"Model is on device: {model.device}")

# 학습 시작
print("Starting training...")
trainer.train()

print("Training completed!")


In [ ]:
from torch.utils.data import DataLoader

# DataLoader 생성 (collate_fn으로 데이터 구조를 맞춤)
data_loader = DataLoader(tokenized_prompts, batch_size=2, collate_fn=data_collator)

# 첫 번째 배치 가져오기
first_batch = next(iter(data_loader))

# 각 텐서의 shape 출력
print("input_ids shape:", first_batch['input_ids'].shape)
print("attention_mask shape:", first_batch['attention_mask'].shape)
print("labels shape:", first_batch['labels'].shape)


In [ ]:
# Trainer 설정 및 학습 시작
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_prompts,  # 토큰화된 데이터셋 사용
    data_collator=data_collator  # data_collator를 사용해 배치 구성
)

# 학습 시작
trainer.train()


In [ ]:
# labels 필드가 제대로 추가되었는지 확인
print(tokenized_prompts[0])


In [ ]:
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

In [ ]:
import torch

def generate_summary(text):
    # 입력 텍스트 토큰화
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # 요약 생성
    summary_ids = model.generate(**inputs, max_length=150, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# 요약할 새로운 데이터 예시
new_data = [
    "여기에 새로운 본문 텍스트를 넣습니다.",
    "다른 요약이 필요한 본문 텍스트를 여기에 넣습니다."
]

# 요약 생성 및 출력
for text in new_data:
    summary = generate_summary(text)
    print("원문:", text)
    print("요약:", summary)
    print("-" * 50)
